<a href="https://colab.research.google.com/github/dlsanf2000/HIR/blob/main/real_classificationreport_i2b2_on_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install transformers
import transformers

     |████████████████████████████████| 2.0MB 17.1MB/s 
     |████████████████████████████████| 890kB 49.1MB/s 
     |████████████████████████████████| 3.2MB 52.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=6b4e938cf1f979b70379fbff4a1af39a79e2a85bf523a74ec45ef80c980b5225
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
from tqdm import tqdm, trange

In [ ]:
import pickle

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sep_df_Train.csv') #Set Dir
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sep_df_Test.csv') #Set Dir

In [ ]:
train_df['words']

0       ['Record', 'date:', '2083-10-18', 'CARDIOLOGY'...
1       ['25', 'years.', 'He', 'is', 'not', 'married,'...
2       ['tracing', 'is', 'Unchanged', 'Selected', 're...
3       ['Record', 'date:', '2081-04-13', 'CARDIOLOGY'...
4       ['history:', 'Unchanged', 'from', 'previous', ...
                              ...                        
2374    ['is', 'soft', 'and', 'non-tender,', 'no', 'sk...
2375    ['due', 'to', 'cardiac', 'stents.', '2.', 'Upo...
2376    ['Record', 'date:', '2074-02-22', 'Gregg,', 'L...
2377    ['cut', 'back', 'because', 'his', 'wife', 'has...
2378    ['I', 'think', 'he', 'found', 'this', 'soberin...
Name: words, Length: 2379, dtype: object

In [ ]:
import ast
train_df['words'] = train_df['words'].apply(ast.literal_eval)
train_df['tags'] = train_df['tags'].apply(ast.literal_eval)
test_df['words'] = test_df['words'].apply(ast.literal_eval)
test_df['tags'] = test_df['tags'].apply(ast.literal_eval)

train_words = train_df.words
train_tags = train_df.tags

test_words = test_df.words
test_tags = test_df.tags

In [ ]:
train_tags[0]

['O',
 'O',
 'B-DATE',
 'O',
 'B-HOSPITAL',
 'I-HOSPITAL',
 'I-HOSPITAL',
 'O',
 'O',
 'O',
 'O',
 'B-PATIENT',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-DATE',
 'O',
 'O',
 'B-HOSPITAL',
 'O',
 'O',
 'O',
 'O',
 'B-HOSPITAL',
 'I-HOSPITAL',
 'I-HOSPITAL',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-DATE',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-DATE',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O

In [ ]:

New_train_words , New_train_tags =  train_df['words'], train_df['tags']
New_test_words , New_test_tags =  test_df['words'], test_df['tags']

In [ ]:
class SentenceGetter(object):

    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]

    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [ ]:
train_sentences = New_train_words
test_sentences = New_test_words
train_labels =  New_train_tags
test_labels = New_test_tags

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataframe.csv') #Set Dir

data

,Unnamed: 0,Word,Tag
0,0,Record,O
1,1,date:,O
2,2,2083-10-18,B-DATE
3,3,CARDIOLOGY,O
4,4,WETZEL,B-HOSPITAL
...,...,...,...
815121,815121,Frank,B-DOCTOR
815122,815122,T.,I-DOCTOR
815123,815123,Xuan,I-DOCTOR
815124,815124,",",O


In [ ]:
tag_values = list(set(data["Tag"].values))
tag_values.append("PAD")
tag2idx = {t: i for i, t in enumerate(tag_values)}

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig

from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

torch.__version__

'1.8.0+cu101'

In [ ]:
MAX_LEN = 256
bs = 32

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [ ]:
torch.cuda.get_device_name(0)


'Tesla T4'

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=False)


In [ ]:
def tokenize_and_preserve_labels(sentence, text_labels):
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

In [ ]:

train_tokenized_texts_and_labels = [
    tokenize_and_preserve_labels(sent, labs)
    for sent, labs in zip(train_sentences, train_labels)
]

test_tokenized_texts_and_labels = [
    tokenize_and_preserve_labels(sent, labs)
    for sent, labs in zip(test_sentences, test_labels)
]

In [ ]:
train_tokenized_texts = [token_label_pair[0] for token_label_pair in train_tokenized_texts_and_labels]
train_labels = [token_label_pair[1] for token_label_pair in train_tokenized_texts_and_labels]

test_tokenize_texts = [token_label_pair[0] for token_label_pair in test_tokenized_texts_and_labels]
test_labels = [token_label_pair[1] for token_label_pair in test_tokenized_texts_and_labels]

In [ ]:
test_tokenize_texts[0][:10], test_labels[0][:10]


(['[UNK]', 'date', ':', '206', '##8', '-', '03', '-', '31', '[UNK]'],
 ['O',
  'O',
  'O',
  'B-DATE',
  'B-DATE',
  'B-DATE',
  'B-DATE',
  'B-DATE',
  'B-DATE',
  'B-HOSPITAL'])

In [ ]:

train_input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in train_tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")

test_input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in test_tokenize_texts],
                          maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")

In [ ]:
train_tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in train_labels],
                     maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
                     dtype="long", truncating="post")

test_tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in test_labels],
                     maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
                     dtype="long", truncating="post")


In [ ]:
train_attention_masks = [[float(i != 0.0) for i in ii] for ii in train_input_ids]
test_attention_masks = [[float(i != 0.0) for i in ii] for ii in test_input_ids]

In [ ]:
tr_inputs, val_inputs, tr_tags, val_tags = train_input_ids, test_input_ids, train_tags, test_tags

tr_masks, val_masks = train_attention_masks, test_attention_masks

In [ ]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [ ]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [ ]:

import transformers
from transformers import BertForTokenClassification, AdamW

transformers.__version__

'4.4.2'

In [ ]:

model = BertForTokenClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=len(tag2idx),
    output_attentions = False,
    output_hidden_states = False
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

In [ ]:
model.cuda();


In [ ]:
input_size = len(train_data)
num_layers = 3

In [ ]:

FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters())
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

optimizer = AdamW(
    optimizer_grouped_parameters,
    lr=3e-5,
    eps=1e-8
)

In [ ]:
from transformers import get_linear_schedule_with_warmup

epochs = 5
max_grad_norm = 1.0

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

In [ ]:

!pip install seqeval==0.0.17
from seqeval.metrics import f1_score, accuracy_score,precision_score,recall_score

  Created wheel for seqeval: filename=seqeval-0.0.17-cp37-none-any.whl size=7640 sha256=8420fae6485590995aa937ce02d5a669f2902e5ccf33d695341b6cd427a40d84
  Stored in directory: /root/.cache/pip/wheels/6a/c8/cf/7b9d5d52984c08ce4d27d6f858a682ef74a3738f43f489166a
Successfully built seqeval


In [ ]:
## 시간 측정 

import time
start = time.time()


## Store the average loss after each epoch so we can plot them.
loss_values, validation_loss_values = [], []

for _ in trange(epochs, desc="Epoch"):
    # ========================================
    #               Training
    # ========================================
    # Perform one full pass over the training set.

    # Put the model into training mode.
    model.train()
    # Reset the total loss for this epoch.
    total_loss = 0

    # Training loop
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # Always clear any previously calculated gradients before performing a backward pass.
        model.zero_grad()
        # forward pass
        # This will return the loss (rather than the model output)
        # because we have provided the `labels`.
        outputs = model(b_input_ids, token_type_ids=None,
                        attention_mask=b_input_mask, labels=b_labels)
        # get the loss
        loss = outputs[0]
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # track train loss
        total_loss += loss.item()
        # Clip the norm of the gradient
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)
    print("Average train loss: {}".format(avg_train_loss))

    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)


    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.
    # Put the model into evaluation mode
    model.eval()
    # Reset the validation loss for this epoch.
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        # Telling the model not to compute or store gradients,
        # saving memory and speeding up validation
        with torch.no_grad():
            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have not provided labels.
            outputs = model(b_input_ids, token_type_ids=None,
                            attention_mask=b_input_mask, labels=b_labels)
        # Move logits and labels to CPU
        logits = outputs[1].detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences.
        eval_loss += outputs[0].mean().item()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.extend(label_ids)

    eval_loss = eval_loss / len(valid_dataloader)
    validation_loss_values.append(eval_loss)
    print("Validation loss: {}".format(eval_loss))
    pred_tags = [tag_values[p_i] for p, l in zip(predictions, true_labels)
                                 for p_i, l_i in zip(p, l) if tag_values[l_i] != "PAD"]
    valid_tags = [tag_values[l_i] for l in true_labels
                                  for l_i in l if tag_values[l_i] != "PAD"]
    print("Validation Accuracy: {}".format(accuracy_score(pred_tags, valid_tags)))
    print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags)))
    print("Validation Recall : {}".format(recall_score(pred_tags, valid_tags)))
    print("Validation Precision : {}".format(precision_score(pred_tags, valid_tags)))
    print()
## 시간 측정 
print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Average train loss: 0.4040211276213328
Validation loss: 0.14268981339409947
Validation Accuracy: 0.967309113650577
Validation F1-Score: 0.7433986567301499
Validation Recall : 0.8093354034156358


Epoch:  20%|██        | 1/5 [02:24<09:38, 144.71s/it]

Validation Precision : 0.687396316304394

Average train loss: 0.12348492989937464
Validation loss: 0.09357537236064672
Validation Accuracy: 0.9770595258400137
Validation F1-Score: 0.8102149950889447
Validation Recall : 0.8320631220299471


Epoch:  40%|████      | 2/5 [04:54<07:18, 146.24s/it]

...

Validation Precision : 0.840954528053086

Average train loss: 0.0693533842265606
Validation loss: 0.06559560735089083
Validation Accuracy: 0.9831144465290806
Validation F1-Score: 0.8529249798788495
Validation Recall : 0.8493988609997891


Epoch:  80%|████████  | 4/5 [09:53<02:27, 147.86s/it]

Validation Precision : 0.8564804968310009

Average train loss: 0.06184330865740776
Validation loss: 0.06369506086533268
Validation Accuracy: 0.9839814656887828
Validation F1-Score: 0.8605248085530642
Validation Recall : 0.8582853757616791


Epoch: 100%|██████████| 5/5 [12:23<00:00, 148.62s/it]

Validation Precision : 0.8627759581436897

time : 743.1000471115112


In [ ]:
# Reset the validation loss for this epoch.
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
predictions , true_labels = [], []
for batch in valid_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    # Telling the model not to compute or store gradients,
    # saving memory and speeding up validation
    with torch.no_grad():
        # Forward pass, calculate logit predictions.
        # This will return the logits rather than the loss because we have not provided labels.
        outputs = model(b_input_ids, token_type_ids=None,
                        attention_mask=b_input_mask, labels=b_labels)
    # Move logits and labels to CPU
    logits = outputs[1].detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Calculate the accuracy for this batch of test sentences.
    eval_loss += outputs[0].mean().item()
    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    true_labels.extend(label_ids)

eval_loss = eval_loss / len(valid_dataloader)
validation_loss_values.append(eval_loss)
print("Validation loss: {}".format(eval_loss))
pred_tags = [tag_values[p_i] for p, l in zip(predictions, true_labels)
                              for p_i, l_i in zip(p, l) if tag_values[l_i] != "PAD"]
valid_tags = [tag_values[l_i] for l in true_labels
                              for l_i in l if tag_values[l_i] != "PAD"]
print("Validation Accuracy: {}".format(accuracy_score(valid_tags,pred_tags)))
print("Validation F1-Score: {}".format(f1_score(valid_tags,pred_tags)))
print()

Validation loss: 0.06369506086533268
Validation Accuracy: 0.9839814656887828
Validation F1-Score: 0.8605248085530642



In [ ]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

print("Validation Accuracy: {}".format(accuracy_score(valid_tags,pred_tags)))
print("precision_score: {}".format(precision_score(valid_tags,pred_tags)))
print("recall_score: {}".format(recall_score(valid_tags,pred_tags)))
print("Validation F1-Score: {}".format(f1_score(valid_tags,pred_tags)))

Validation Accuracy: 0.9839814656887828
precision_score: 0.8582853757616791
recall_score: 0.8627759581436897
Validation F1-Score: 0.8605248085530642


In [ ]:
print(classification_report(valid_tags, pred_tags))

                precision    recall  f1-score   support

           AGE       0.85      0.94      0.89       674
          CITY       0.69      0.11      0.19       221
       COUNTRY       0.00      0.00      0.00        95
          DATE       0.98      0.98      0.98     14573
        DEVICE       0.00      0.00      0.00        14
        DOCTOR       0.44      0.57      0.50      1661
      HOSPITAL       0.32      0.31      0.31       751
         IDNUM       0.88      0.93      0.90       656
LOCATION-OTHER       0.00      0.00      0.00        11
 MEDICALRECORD       0.98      0.97      0.98      1949
  ORGANIZATION       0.00      0.00      0.00        67
       PATIENT       0.53      0.49      0.51      1192
         PHONE       0.75      0.91      0.82       715
    PROFESSION       0.00      0.00      0.00       135
         STATE       0.92      0.68      0.78       167
        STREET       0.12      0.13      0.12       169
      USERNAME       0.00      0.00      0.00  